# 0. Imports

In [8]:
import requests
import pandas as pd
import numpy as np
import math
import re 
import numpy as np


from datetime import datetime
from bs4 import BeautifulSoup

# 1. Function

In [9]:
#initialization of coding 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) , AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}

def webscraping_glassdoor(url):
    #===========================URL-PARA O WEBSCRAPING===============================================
    page = requests.get( url, headers=headers )
    soup = BeautifulSoup( page.text, "html.parser")
    
    #==========================01.EXTRAIR O NOME DA EMRPESA==========================================
    #nome da empresa fica em h3 com a especificação "data-test"
    nome_ = soup.find_all("h3", 
                               {"data-test": re.compile("salaries-list-item-.*-employer-name")})
  
    #salva o nome da empresa em uma lista
    nome = []
    for i in nome_: 
             nome.append(i.find("a").text)
 

            
    #=========================02.EXTRAIR SALÁRIO DA EMPRESA===========================================
    #o salário fica em h3 na classe m-0 css-g261rn
    sal = soup.find_all("h3", class_="m-0 css-g261rn")
    
    #extraimos para uma lista 
    price_classic = [p.get_text() for p in sal]
    
    #declaramos algumas listas para armazenarem os salários ---> lembrando que temos 2 tipos de salário (menor e maior salário)
    salario = []
    salario1 = []
    salario2 = []
    i = 1 

    #varredura para salvar os valores de salário
    for i in range(len(price_classic)):
            if (i % 2) != 0:
                salario.append(price_classic[i])

    #salva os valores em uma lista -> podendo ter duas ou uma casas (salário mínimo e máximo do cargo)
    for p in range(len(salario)):        
                salario[p] = [float(s) for s in re.findall(r'-?\d+\.?\d*', salario[p])]
                      
    #varredura para salvar os valores como milhar --> tendo em vista que esse é o valor real 
    var_ = 0
    for x in range(len(salario)): 
                var_ = len(salario[x])

                if var_ == 2:
                    salario1.append(salario[x][0] *1000)
                    salario2.append(salario[x][1] *1000)
                elif var_==1: 
                    salario1.append(salario[x][0] *1000)
                    salario2.append("NA")
            
    

    #========================03. EXTRAIR O CARGO (EXEMPLO: ESTAGIÁRIO, JR, LEAD)=======================
    #cargos fica em uma div (divisão) com classe "col-12 col-lg px-xsm"
    cargos = soup.find_all("div", class_="col-12 col-lg px-xsm")
    cargos_ = [p.get_text() for p in cargos]

    

    
    #===========================SALVAMOS OS VALORES EM UM DATAFRAME========================================
    data = pd.DataFrame([nome, salario1, salario2, cargos_]).T
    data.columns = ["Nome da Empresa", "Mínimo Salário", "Máximo Salário", "Cargo"]
    
    
    return data 


def juntar_df(pagetotal):
    
    for i in range(len(pagetotal)-1):
        #data = pd.merge(pagetotal[i], pagetotal[i+1], how = 'outer')
        pagetotal[0]  = pagetotal[0].append(pagetotal[i+1],ignore_index=True)
        
    return pagetotal[0]



# 3. Extraindo salários de People Analytics

In [10]:
page1 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/analyst-people-analytics-sal%C3%A1rio-SRCH_KO0,24.htm")
page2 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/analyst-people-analytics-sal%C3%A1rio-SRCH_KO0,24_IP2.htm")

pagetotal = [page1, page2]
people_analytics = juntar_df(pagetotal)

In [11]:
people_analytics

,Nome da Empresa,Mínimo Salário,Máximo Salário,Cargo
0,Dasa,8000.0,8000.0,Analyst People Analytics: prestador de serviço...
1,Nubank,9000.0,9000.0,Analyst People Analytics: mensal
2,QuintoAndar,4000.0,5000.0,Analyst People Analytics: mensal
3,Catho Online,5000.0,6000.0,Analyst People Analytics: mensal
4,NTCONSULT,6000.0,7000.0,Analyst People Analytics: prestador de serviço...
5,Sistema Ocemg,5000.0,6000.0,Analyst People Analytics: mensal
6,RD Station,5000.0,6000.0,People Analytics: mensal
7,EBANX,2000.0,2000.0,People Analytics: estagiário mensal
8,Boa Vista Serviços,4000.0,4000.0,People Analytics: mensal
9,CHS Inc,8000.0,8000.0,People Analytics: mensal


In [17]:
people_analytics.to_csv("People_Analytics-Salary.csv")

# 4. Extraindo salários de Data Science 

In [13]:
#Paginas de Data Science que eu gostaria de extrair um relatório
page1 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/data-scientist-sal%C3%A1rio-SRCH_KO0,14.htm")
page2 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/data-scientist-sal%C3%A1rio-SRCH_KO0,14_IP2.htm")
page3 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/data-scientist-sal%C3%A1rio-SRCH_KO0,14_IP3.htm")
page4 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/data-scientist-sal%C3%A1rio-SRCH_KO0,14_IP4.htm")
page5 = webscraping_glassdoor("https://www.glassdoor.com.br/Sal%C3%A1rios/data-scientist-sal%C3%A1rio-SRCH_KO0,14_IP5.htm")

#agrupamos os dataframes em uma lista
pagetotal = [page1, page2, page3, page4, page5]

#juntamos todos os dataframes em somente um 
data_science = juntar_df(pagetotal)

In [14]:
data_science

,Nome da Empresa,Mínimo Salário,Máximo Salário,Cargo
0,Itaú Unibanco (Itaú BBA e Rede),8963.0,NA,Data Scientist: mensal
1,Nubank,14341.0,NA,Data Scientist: mensal
2,IBM,9345.0,NA,Data Scientist: mensal
3,Loft,12359.0,NA,Data Scientist: mensal
4,Creditas,8698.0,NA,Data Scientist: mensal
...,...,...,...,...
95,Soko (Brazil),5000.0,5000.0,Data Scientist: mensal
96,Optimum Soluções Estratégicas,7000.0,9000.0,Data Scientist: mensal
97,Dafiti,6000.0,12000.0,Data Scientist: mensal
98,FALCONI,11000.0,18000.0,Data Scientist: mensal


In [18]:
data_science.to_csv("Data_Science-Salary.csv")